In [1]:
import os
import ast
import json
import warnings
import pandas as pd
from qdrant_client import models, QdrantClient

In [2]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_community.vectorstores import Qdrant
from langchain.retrievers.self_query.qdrant import QdrantTranslator
from langchain_openai import OpenAI, ChatOpenAI

In [3]:


handler = ConsoleCallbackHandler()
llm = ChatOpenAI(temperature=0, model="gpt-4o")
#llm = OpenAI(temperature=0)
client = QdrantClient(
        url="http://localhost:6333"
)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vectorstore = Qdrant(client, collection_name="apartments", embeddings=embeddings)

/tmp/ipykernel_69702/2188913810.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
/home/tavera/projects/properties_web/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_69702/2188913810.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in th

In [4]:
qdrant_translator = QdrantTranslator(metadata_key="metadata")


In [19]:
metadata_field_info = [
    AttributeInfo(
        name="bedrooms",
        description="El numero de habitaciones en el apartamento",
        type="integer",
    ),
    AttributeInfo(
        name="bathrooms",
        description="El numero de baños en el apartamento",
        type="integer",
    ),
    AttributeInfo(
        name="price",
        description="El costo de la renta del apartamento en pesos colombianos",
        type="float",
    ),
    AttributeInfo(
        name="area",
        description="El area del apartamento en metros cuadrados",
        type="integer",

    ),
    AttributeInfo(
        name="agency",
        description="Nombre de la inmobiliaria que ofrece el apartamento",
        type="string",

    ),
    AttributeInfo(
        name="facilities",
        description="Todas las comodidades que tiene el apartamento (piscina, gimnasio, etc.)",
        type="list[string]",

    ),
    AttributeInfo(
        name="stratum",
        description="El estrato del apartamento (1-6) donde 1 es el mas bajo y 6 el mas alto",
        type="integer",

    ),

    AttributeInfo(
        name="parking_lots",
        description="El numero de parqueaderos que tiene el apartamento",
        type="integer",

    ),
    AttributeInfo(
        name="floor",
        description="El numero del piso donde se encuentra el apartamento, si es -1 no sabemos donde esta",
        type="integer",

    ),

    AttributeInfo(
        name="upload_date",
        description="La fecha de subida del apartamento a la plataforma",
        type="string",

    ),

    AttributeInfo(
        name="construction_age_min",
        description="Años minimos de construccion del apartamento, Ej: lleva al menos 1 año construido",
        type="integer",

    ),
    AttributeInfo(
        name="construction_age_max",
        description="Años maximos de construccion del apartamento, Ej: lleva menos de 8 años de construido",
        type="integer",

    ),
    AttributeInfo(
        name="places",
        description="Los lugares cercanos al apartamento (universidades, parques, etc.)",
        type="list[string]",

    ),
    AttributeInfo(
        name="location",
        description="El barrio, ciudad y departamento donde se encuentra el apartamento",
        type="list[string]",

    ),
    AttributeInfo(
        name="transportation",
        description="Las vias de transporte cercanas al apartamento (transmilenio, paradas, etc.)",
        type="list[string]",

    ),

]

document_content_description = "Breve descripcion del apartamento enfocandose en la iluminacion, vista, acabados, etc. " \

retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    structured_query_translator=qdrant_translator,
    search_kwargs={"k": 10}, 
    # you can also pass chain_kwargs, enable_limit, use_original_query here if needed
)

In [28]:
response = retriever.invoke("Apartamentos cerca a la Universidad de los Andes")
response

[Document(metadata={'link': 'https://www.fincaraiz.com.co/apartamento-en-arriendo/192405585', 'price': 11500000, 'bedrooms': 2, 'bathrooms': 3, 'area': 160, 'agency': 'Engel & Völkers', 'coordinates': '(4.638836065010859, -74.05976057052612)', 'facilities': ['Terraza', 'Balcón', 'Duplex', 'Zona Infantil', 'Chimenea', 'Ascensor', 'Jardín', 'Salón Comunal', 'Gimnasio', 'Alarma', 'Amoblado'], 'upload_date': '2025-05-18T00:00:00', 'stratum': 4, 'parking_lots': 2, 'floor': 8, 'construction_age_min': 16, 'construction_age_max': 30, 'places': ['Parque de los Hippies', 'Club THEATRON', 'Éxito Chapinero', 'La Pinta de Bogotá', 'Mistral', 'Insurgentes - Taco Bar', 'Little Caesars Javeriana', 'El Mono Bandido Chapinero', 'Mesa Franca', 'Mini Mal', 'Restaurante Salvo Patria', 'Restaurante Río', 'Hornitos Calle 51', 'Flora', 'Tejo Turmequé', 'Carulla Calle 47', 'Indigo Club', "McDonald's", 'Amen Ramen', 'El Pantera Taqueria'], 'location': ['Bogotá', 'Cundinamarca', 'Chapinero'], 'transportation': [

In [30]:
# 1) Asume que ya tienes esto configurado:
#    - `retriever` = tu SelfQueryRetriever
#    - `llm`       = ChatOpenAI(...)
#    - necesarias: langchain, langchain-community, qdrant-client, python-dotenv, lark-parser, etc.

from langchain.memory import ConversationBufferMemory
from langchain.agents import Tool, initialize_agent, AgentType

# 2) Estado global para almacenar resultados de búsqueda
last_results = []

# 3) Herramienta de búsqueda
def search_apartments(query: str) -> str:
    """
    Busca apartamentos que coincidan con la consulta,
    almacena la lista en `last_results` y devuelve un resumen numerado.
    """
    global last_results
    docs = retriever.get_relevant_documents(query)
    last_results = docs

    if not docs:
        return "No encontré ningún apartamento para esa consulta."

    lines = []
    for i, doc in enumerate(docs, start=1):
        md = doc.metadata
        lines.append(
            f"{i}. {md.get('agency','')} — {md.get('bedrooms','?')} hab., "
            f"{md.get('bathrooms','?')} baños, precio: {md.get('price','?')}"
        )
    return "\n".join(lines)

# 4) Herramienta de detalle
def get_apartment_details(selection: str) -> str:
    """
    Dado un número (‘1’, ‘2’, etc.), devuelve todos los metadatos del apartamento.
    """
    if not last_results:
        return "Primero realiza una búsqueda para guardar resultados."
    try:
        idx = int(selection.strip()) - 1
    except ValueError:
        return "Por favor indica un número válido, p.e. ‘2’ para el segundo."

    if idx < 0 or idx >= len(last_results):
        return f"Sólo hay {len(last_results)} resultados; elige un número entre 1 y {len(last_results)}."

    md = last_results[idx].metadata
    # Construimos una respuesta legible
    details = [f"{key}: {value}" for key, value in md.items()]
    return "Detalles del apartamento seleccionado:\n" + "\n".join(details)

# 5) Wrap tools
search_tool = Tool(
    name="search_apartments",
    func=search_apartments,
    description="Busca apartamentos según la consulta y devuelve un listado numerado."
)
detail_tool = Tool(
    name="get_apartment_details",
    func=get_apartment_details,
    description="Devuelve los metadatos completos del apartamento indicado por su número."
)

# 6) Memoria de la conversación
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 7) Inicializar el agente
agent = initialize_agent(
    tools=[search_tool, detail_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# 8) Ejemplo de uso:
print(agent.run("Busca apartamentos de menos de 2 millones de pesos con 2 habitaciones en Chapinero"))  
# → muestra listado numerado

print(agent.run("Me gusta el segundo"))  
# → llama a get_apartment_details("2") y devuelve los metadatos

print(agent.run("¿Cuál es su valor de arriendo?"))  
# → como está en memoria, el agente se refiere al mismo apartamento y responde




> Entering new AgentExecutor chain...
To find apartments that meet the criteria, I need to search for apartments in Chapinero with a price under 2 million pesos and 2 bedrooms. 

Action: search_apartments
Action Input: "apartamentos en Chapinero con 2 habitaciones por menos de 2 millones de pesos"
Observation: 1. PORTADA INVERSIONES FONTIBON — 2 hab., 1 baños, precio: 1700000
Thought:The search returned a result for an apartment in Chapinero that meets the criteria of having 2 bedrooms and a price under 2 million pesos. I will now get more details about this apartment.

Action: get_apartment_details
Action Input: "1"
Observation: Detalles del apartamento seleccionado:
link: https://www.fincaraiz.com.co/apartamento-en-arriendo/192399747
price: 1700000
bedrooms: 2
bathrooms: 1
area: 42
agency: PORTADA INVERSIONES FONTIBON
coordinates: (4.644674796418426, -74.11250352859497)
facilities: ['Salón Comunal', 'Salón de Juegos', 'Supermercados / C.Comerciales', 'En conjunto cerrado', 'Sobre v